<a href="https://colab.research.google.com/github/sriharikrishna/EuroAD26/blob/main/EuroAD_scipy_odetest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The initial code was written in SciPy. The ODE solver is implemented through ```scipy.integrate.solve_ivp```



In [ ]:
import numpy as np
import scipy

def solver(y0, t_f, A):
  def ode_fn(_, y, B):
    return np.matmul(B[0],y)

  solution = scipy.integrate.solve_ivp(
        ode_fn,
        [0, t_f],
        y0,
        t_eval=[t_f],
        method="DOP853",
        args=(A,),
    )
  final_vec = solution.y[:, -1]
  return final_vec.reshape(y0.shape)


Let us now call the solver

In [ ]:
t_f = np.array(2., dtype=np.float64)

y0 = np.array([1.0, 9.0], dtype=np.float64)

A = np.array([[0, 1.0],
               [- 100.0, 0]], dtype=np.float64)

print(solver(y0, t_f, A))

Changing the datatype to ```complex128``` results in a failure.

In [ ]:
t_f = np.array(2., dtype=np.complex128)

y0 = np.array([1.0, 9.0], dtype=np.complex128)

A = np.array([[0, 1.0],
               [- 100.0, 0]], dtype=np.complex128)

print(solver(y0, t_f, A))

There is a way out. We can maintain the time variable to be ```float64``` type.

In [ ]:
t_f = np.array(2., dtype=np.float64)

y0 = np.array([1.0, 9.0], dtype=np.complex128)

A = np.array([[0, 1.0],
               [- 100.0, 0]], dtype=np.complex128)

print(solver(y0, t_f, A))

The HIPS Autograd package can be used to provide AD capability for Scipy. We simply have to change which NumPy and SciPy packages that we already use.

In [ ]:
import autograd
import autograd.numpy as np
import autograd.scipy as scipy

def solver(y0, t_f, A):
  t = np.linspace(0., t_f, 10)
  def ode_fn(_, y, B):
    return np.matmul(B[0],y)

  solution = scipy.integrate.solve_ivp(
        ode_fn,
        [0, t_f],
        y0,
        t_eval=[t_f],
        method="DOP853",
        args=(A,),
    )
  final_vec = solution.y[:, -1]
  return final_vec.reshape(y0.shape)

But Autograd does not support ```scipy.integrate.solve_ivp```. This was a problem for the Physicists!

In [ ]:
t_f = np.array(2., dtype=np.float64)

y0 = np.array([1.0, 9.0], dtype=np.float64)

A = np.array([[0, 1.0],
               [- 100.0, 0]], dtype=np.float64)

print(solver(y0, t_f, A))

Autograd instead supports ```scipy.integrate.odeint```



In [ ]:
import autograd
import autograd.numpy as np
import autograd.scipy as scipy

def solver(y0, t_f, A):
  def ode_fn(y, _, B):
    return np.matmul(B,y)
  #Solve a system of ordinary differential equations using lsoda from the FORTRAN library odepack.
  solution = scipy.integrate.odeint(
        ode_fn,
        y0,
        [0, t_f],
        (A,),
    )
  final_vec = solution[-1]
  return final_vec.reshape(y0.shape)

Let us call it. I still need to figure out how to make the two answers match!

In [ ]:
t_f = np.array(2., dtype=np.float64)

y0 = np.array([1.0, 9.0], dtype=np.float64)

A = np.array([[0, 1.0],
               [- 100.0, 0]], dtype=np.float64)

print(solver(y0, t_f, A))

We can now differentiate ```odeint```!

In [ ]:
from autograd import grad, jacobian

print(jacobian(solver)(y0, t_f, A))


Let us try some ```complex valued``` input

In [ ]:
t_f = np.array(2., dtype=np.float64)

y0 = np.array([1.0, 9.0], dtype=np.complex128)

A = np.array([[0, 1.0],
               [- 100.0, 0]], dtype=np.complex128)

print(solver(y0, t_f, A))

Conclusion: Look elsewhere!